# BLIP 모델을 이용한 이미지의 색상 정보 추출.

### 문제 배경
한글 상표 생성 데이터셋은 이미지와 이미지에 대한 한글 설명이 담긴 txt 파일로 이루어져 있다. txt파일에는 해당 상표 이미지의 배경색상과 글자색에 대한 정보를 입력해줘야 했다. 그래서 AIHub의 '야외 실제 촬영 한글 이미지' 데이터셋에서 받은 '책 표지'데이터를 상표 생성 데이터셋에 맞게 수정하여 활용하고자 했다. 그런데 AIHub 데이터셋의 어노테이션에는 이미지의 배경색상과 글자색에 대한 정보가 없었고, 이를 해결하고자 BLIP 모델을 이용하여 image captioning 작업을 수행하여 해결하고자 했다.

#### 문제1 : image Captioning은 색상에 대한 정보를 뽑아 주지 않았다.
BLIP 모델을 사용하여 image captioning을 진행해 본 결과, 이미지 내의 사물의 유무에 대한 설명만 있을 뿐, 색상에 대해서는 정보를 제공해주지 않았다.
이를 해결하고자, Visual question answering (VQA) task로 문제를 재정의하여 색상에 대한 질문을 주면 답을 얻는 방식으로 문제를 해결했다.

#### 문제2 : VQA모델을 통해 얻은 색상에 대한 정보는 영어이다.
VQA model로 얻은 색상 정보는 영어로 작성되어 있다. 이를 해결하기 위해 두 가지 방법을 사용했다.
- 첫 번째 방법은 구글 번역기 API를 활용하는 것이다. 구글 번역기 api를 이용해서 한글로 번역하고 txt 파일을 작성했었는데, api 요청시 중간중간 오류가 발생해서, 오류를 수정하기 번거러웠다. 구글 번역기 api : https://py-googletrans.readthedocs.io/en/latest/
- 두 번째 방법은, VQA에 모든 데이터셋을 통과시켜서 색상 카테고리 리스트를 만들고, {영어:한글}을 직접 번역하여 매칭시킨 dict를 이용해서 한글로 번역하고 txt 파일로 저장했다. VQA 모델 깃허브 : https://github.com/salesforce/LAVIS

#### 코드 설명
AIHub 데이터셋에서 필요한 정보만 빼서 미리 만들어 놓은 기존 txt파일에 색상 정보를 추가한 새로운 txt 파일을 만드는 코드를 작성했다.

In [ ]:
import os
from tqdm import tqdm

from PIL import Image
import torch
from lavis.models import load_model_and_preprocess

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Load BLIP VQA model finetuned on VQAv2
model, vis_processors, txt_processors = load_model_and_preprocess(name="blip_vqa", model_type="vqav2", is_eval=True, device=device)

In [5]:
batch_size = 2

# 배치 단위의 질문을 만든다. 이미지 수와 질문 수를 일치 시켜야한다.
question_1 = txt_processors["eval"]("what colors are the background of this picture?")
question_2 = txt_processors["eval"]("which colors is the letters of this photo?")

def color_extractor(image_path):
    raw_image = Image.open(image_path).convert("RGB")
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)
    
    # 샘플을 배치 수 만큼 만든다. 다양한 이미지 혹은 같은 이미지로 구성할 수 있다.
    image_batch = image.repeat(batch_size, 1, 1, 1)

    question_batch = [question_1, question_2]
    colors = model.predict_answers(samples={"image": image_batch, "text_input": question_batch}, inference_method="generate")
    colors = (colors[0].lower(), colors[1].lower())
    return colors

In [ ]:
"""
이미지 데이터셋 내에 있는 색상 카테고리를 만드는 코드
"""
# AIHub 이미지 파일이 있는 폴더의 주소
img_files_path = "../selected_data/selected_images"

color_dict = []
img_files_list = os.listdir(img_files_path)

i = 0
for img_file in tqdm(img_files_list):
    # 짝수 번째의 이미지의 컬러 추출
    if i%2 == 0:
        i += 1
        continue

    img_path = os.path.join(img_files_path, img_file)
    colors = color_extractor(img_path)
    for color in colors[0].split()+colors[1].split():
        if color not in color_list and color != "and":
            color_list.append(color)
    i += 1

print(color_list)

In [11]:
def color2str(mode, colors):
    global check_list, exception_list, color_dict
    cl = []
    if mode == 'background':
        annotation = 'bg: '
        colors = colors[0].split()
    elif mode == 'letter':
        annotation = 'letter: '
        colors = colors[1].split()

    for color in colors:
        if color in color_dict and color != "and":
            cl.append(color_dict[color])
            if color in ['multicolored', 'multi', 'colored']:
                check_list.append(annotation+label)
        elif color != "and":
            exception_list.append(annotation+label)
    color_str = ', '.join(cl)
    return color_str

In [ ]:
# AIHub 이미지 파일이 있는 폴더의 주소
img_files_path = "../selected_data/selected_images"
# 기존 txt 파일이 있는 폴더의 주소
label_path = '../selected_data/txt'
# 새롭게 생성한 txt 파일을 저장할 폴더의 주소
output_label_path = '../selected_data/labels_v2'


color_dict = {
    'white':'흰색', 'red':'빨간색', 'green':'초록색', 'black':'검정색', 'blue':'파란색', 'orange':'주황색', 'gold':'금색', 'gray':'회색', 
    'brown':'갈색', 'yellow':'노란색', 'purple':'보라색', 'multicolored':'다양한 색', 'pink':'분홍색', 'teal':'청록색', 'tan':'황토색', 
    'silver':'은색', 'multi':'다양한', 'colored':'색', 'rainbow':'무지개색', 'peach':'복숭아색', 'beige':'베이지색', 'grey':'회색'}

# exception_list는 color_dict에 없는 색상일 경우 해당 이미지의 파일명을 저장
exception_list = []
# check_list는 'multicolored', 'multi', 'colored'로 분류되는 이미지 파일명을 저장
check_list = []

img_files_list = os.listdir(img_files_path)

for img_file in tqdm(img_files_list):
    label = img_file.split('.')[0]+'.txt'
    if os.path.exists(os.path.join(output_label_path, label)):
        continue
    
    img_path = os.path.join(img_files_path, img_file)
    colors = color_extractor(img_path)
    bg_color_str = color2str('background', colors)
    letter_color_str = color2str('letter', colors)
    
    res = f' 로고에서 배경색은 {bg_color_str}이며 글자의 색상은 {letter_color_str}입니다. '
    # 기존의 txt 파일의 내용을 불러온다.
    with open(os.path.join(label_path, label), 'r') as f:
        txt = f.read().strip()
    
    # 기존의 txt파일 내용에 맞춰 문장을 합치고, 새로운 txt파일을 생성한다.
    with open(os.path.join(output_label_path, label), 'w') as f:
        f.write(txt+res)

In [ ]:
# 'multicolored', 'multi', 'colored' 라는 색상으로 추출된 txt 파일 리스트
# bg는 background에서 해당 색상으로 추출됐다는 것을 의미. lc는 글자색.
check_list
# 색상 카테고리 외의 색깔이 나온 예외 파일 리스트
exception_list

In [43]:
# check_list를 txt파일로 만듦.
with open(os.path.join('/workspace/logo_gen_project/check_list.txt'), 'w') as f:
    for s in check_list:
        f.write(s+'\n')